### Load in the training data

In [21]:
import pandas as pd
import numpy as np

# Use read_excel and use participant_id as the index
train_cat = pd.read_excel(
    "../../widsdatathon2025/TRAIN_NEW/TRAIN_CATEGORICAL_METADATA_new.xlsx")
train_func = pd.read_csv("../../widsdatathon2025/TRAIN_NEW/"
                         "TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv")
train_quant = pd.read_excel(
    "../../widsdatathon2025/TRAIN_NEW/TRAIN_QUANTITATIVE_METADATA_new.xlsx")
train_soln = pd.read_excel(
    "../../widsdatathon2025/TRAIN_NEW/TRAINING_SOLUTIONS.xlsx")

In [23]:
# Set index for merging
train_cat.set_index("participant_id", inplace=True)
train_quant.set_index("participant_id", inplace=True)
train_func.set_index("participant_id", inplace=True)
train_soln.set_index("participant_id", inplace=True)

# train_cat.head()
# train_func.head()
# train_quant.head()
# train_soln.head()

KeyError: "None of ['participant_id'] are in the columns"

In [24]:
train_func

,0throw_1thcolumn,0throw_2thcolumn,0throw_3thcolumn,0throw_4thcolumn,0throw_5thcolumn,0throw_6thcolumn,0throw_7thcolumn,0throw_8thcolumn,0throw_9thcolumn,0throw_10thcolumn,...,195throw_196thcolumn,195throw_197thcolumn,195throw_198thcolumn,195throw_199thcolumn,196throw_197thcolumn,196throw_198thcolumn,196throw_199thcolumn,197throw_198thcolumn,197throw_199thcolumn,198throw_199thcolumn
participant_id,,,,,,,,,,,,,,,,,,,,,
70z8Q2xdTXM3,0.222930,0.527903,0.429966,0.060457,0.566489,0.315342,0.508408,-0.078290,0.525692,0.470063,...,0.224985,0.397448,0.422966,0.184642,0.305549,0.420349,0.016328,0.561864,0.471170,0.365221
WHWymJu6zNZi,0.614765,0.577255,0.496127,0.496606,0.404686,0.439724,0.122590,-0.085452,0.120673,0.276350,...,0.217546,-0.014549,0.000440,-0.096451,0.454501,0.343916,0.167313,0.607656,0.550623,0.503176
4PAQp1M6EyAo,-0.116833,0.458408,0.260703,0.639031,0.769337,0.442528,0.637110,0.192010,0.520379,0.378557,...,0.342487,-0.021141,-0.037836,0.075069,0.412712,0.292708,0.391005,0.461544,0.508912,0.624232
obEacy4Of68I,0.199688,0.752714,0.658283,0.575096,0.692867,0.645789,0.522750,0.412188,0.530843,0.259596,...,0.103562,-0.178313,0.210983,-0.018666,0.436313,0.592982,0.216205,0.341272,0.440313,0.558193
s7WzzDcmDOhF,0.227321,0.613268,0.621447,0.562673,0.736709,0.589813,0.266676,0.359668,0.300771,0.331445,...,-0.164956,0.007064,-0.120904,-0.488095,0.493575,-0.215361,0.210685,0.055850,0.119065,0.108273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9gpepMI9sj5q,0.265284,0.551726,0.628318,0.647700,0.710757,0.132334,0.326207,0.354319,0.531170,0.340388,...,0.127097,0.129787,0.298987,0.114577,0.534109,0.118893,0.181292,0.181055,0.238357,0.577009
FIDen5rdMc0v,-0.018377,0.576689,0.527451,0.327463,0.586868,0.573689,0.300544,0.301103,0.659840,0.434427,...,0.482214,-0.159587,-0.162498,-0.093249,0.309914,0.143818,0.218337,0.389331,0.328741,0.238443
dlsMC4TXL4e8,0.227028,0.405659,0.023545,-0.093085,-0.068960,0.647574,0.762552,0.465109,0.199337,-0.357953,...,-0.379304,0.126976,0.193695,0.214483,0.429836,0.302141,0.104774,0.856375,0.303248,0.363639


## PCA for the functional connectome matrix to reduce dimension

In [3]:
from sklearn.decomposition import PCA

# Select only fMRI columns
fmri_data = train_func.drop(columns=['participant_id'], errors='ignore')

# Apply PCA to reduce to top N components (e.g. 50)
pca = PCA(n_components=50)
fmri_pca = pca.fit_transform(fmri_data)

train_func_pca = pd.DataFrame(fmri_pca, index=train_func.index,
                           columns=[f'fmri_pca_{i}' for i in range(1, 51)])
train_func_pca.head()

,fmri_pca_1,fmri_pca_2,fmri_pca_3,fmri_pca_4,fmri_pca_5,fmri_pca_6,fmri_pca_7,fmri_pca_8,fmri_pca_9,fmri_pca_10,...,fmri_pca_41,fmri_pca_42,fmri_pca_43,fmri_pca_44,fmri_pca_45,fmri_pca_46,fmri_pca_47,fmri_pca_48,fmri_pca_49,fmri_pca_50
participant_id,,,,,,,,,,,,,,,,,,,,,
70z8Q2xdTXM3,6.322800,-0.365680,-4.918161,-3.823232,-5.706923,-0.875162,1.567530,-1.347508,-1.717423,1.626893,...,0.297614,0.753887,0.131417,-0.673929,2.742455,-4.644635,0.882842,0.345077,-0.807577,-0.833189
WHWymJu6zNZi,5.468294,-3.985252,1.542399,0.104717,-0.529034,-2.951968,0.786364,0.794950,-2.772031,-2.408772,...,1.083195,-0.565174,-0.865164,-0.010423,-2.180346,-0.462796,-1.177160,-0.183711,-0.411178,1.614361
4PAQp1M6EyAo,0.447190,0.733000,3.051331,1.763901,1.932821,6.451158,-3.356873,1.946055,-1.165651,-2.876763,...,-2.385806,0.175190,2.185090,0.656177,-0.046917,-2.072229,-1.420524,1.101046,-1.340174,2.992421
obEacy4Of68I,-9.149799,-2.044626,0.519359,-2.703924,-4.444127,2.916930,1.036388,0.080339,-0.148149,0.384162,...,-0.169952,1.633509,0.578319,-0.607735,-0.821945,-0.808040,-0.109338,-0.253096,0.441194,-0.409980
s7WzzDcmDOhF,0.812814,-1.933265,1.834524,0.383157,-0.512338,2.972217,3.151061,3.419844,0.283717,-2.997755,...,2.802717,2.170408,-2.237547,-0.435744,-0.823936,1.265570,-0.043517,2.269485,-0.268276,-0.199811


## Interpolate missing values with mean for quant and cat dfs

In [4]:
# Check missing values *before* merging
missing = train_quant.isnull().sum()
missing_cat = train_cat.isnull().sum()
print("Missing values in train_quant:\n", missing[missing > 0])
print("Missing values in train_cat:\n", missing_cat[missing_cat > 0])

# Define columns with small gaps to impute
small_gap_cols = [
    'EHQ_EHQ_Total', 'ColorVision_CV_Score',
    'APQ_P_APQ_P_CP', 'APQ_P_APQ_P_ID', 'APQ_P_APQ_P_INV',
    'APQ_P_APQ_P_OPD', 'APQ_P_APQ_P_PM', 'APQ_P_APQ_P_PP',
    'SDQ_SDQ_Conduct_Problems', 'SDQ_SDQ_Difficulties_Total',
    'SDQ_SDQ_Emotional_Problems', 'SDQ_SDQ_Externalizing',
    'SDQ_SDQ_Generating_Impact', 'SDQ_SDQ_Hyperactivity',
    'SDQ_SDQ_Internalizing', 'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Prosocial',
    'PreInt_Demos_Fam_Child_Ethnicity', 'PreInt_Demos_Fam_Child_Race',
    'MRI_Track_Scan_Location'
]

small_gap_cols_cat = [
    'Barratt_Barratt_P1_Edu', 'Barratt_Barratt_P1_Occ'
]

# Fill small gaps using column means
for col in small_gap_cols:
    if col in train_quant.columns:
        train_quant[col] = train_quant[col].fillna(train_quant[col].mean())
for col in small_gap_cols_cat:
    if col in train_cat.columns:
        train_cat[col] = train_cat[col].fillna(train_cat[col].mean())

train_quant.drop(columns=['MRI_Track_Age_at_Scan'], inplace=True) # Dropped
train_cat.drop(columns=['Barratt_Barratt_P2_Occ', 'Barratt_Barratt_P2_Edu'], 
              inplace=True) # Dropped

Missing values in train_quant:
 EHQ_EHQ_Total                  13
ColorVision_CV_Score           23
APQ_P_APQ_P_CP                 12
APQ_P_APQ_P_ID                 12
APQ_P_APQ_P_INV                12
APQ_P_APQ_P_OPD                12
APQ_P_APQ_P_PM                 12
APQ_P_APQ_P_PP                 12
SDQ_SDQ_Conduct_Problems        9
SDQ_SDQ_Difficulties_Total      9
SDQ_SDQ_Emotional_Problems      9
SDQ_SDQ_Externalizing           9
SDQ_SDQ_Generating_Impact       9
SDQ_SDQ_Hyperactivity           9
SDQ_SDQ_Internalizing           9
SDQ_SDQ_Peer_Problems           9
SDQ_SDQ_Prosocial               9
MRI_Track_Age_at_Scan         360
dtype: int64
Missing values in train_cat:
 PreInt_Demos_Fam_Child_Ethnicity     43
PreInt_Demos_Fam_Child_Race          54
MRI_Track_Scan_Location               3
Barratt_Barratt_P1_Edu               15
Barratt_Barratt_P1_Occ               31
Barratt_Barratt_P2_Edu              198
Barratt_Barratt_P2_Occ              222
dtype: int64


In [5]:
# Check missing values *before* merging
missing = train_quant.isnull().sum()
print("Missing values in train_quant:\n", missing[missing > 0])

Missing values in train_quant:
 Series([], dtype: int64)


In [6]:
train_quant.head()

,EHQ_EHQ_Total,ColorVision_CV_Score,APQ_P_APQ_P_CP,APQ_P_APQ_P_ID,APQ_P_APQ_P_INV,APQ_P_APQ_P_OPD,APQ_P_APQ_P_PM,APQ_P_APQ_P_PP,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial
participant_id,,,,,,,,,,,,,,,,,
00aIpNTbG5uh,100.00,13.0,3.0,15.0,44.0,14.0,20.0,27.0,3.0,17.0,4.0,11.0,5.0,8.0,6.0,2.0,9.0
00fV0OyyoLfw,92.27,14.0,3.0,12.0,35.0,25.0,28.0,30.0,5.0,20.0,4.0,13.0,5.0,8.0,7.0,3.0,8.0
04X1eiS79T4B,86.67,14.0,3.0,21.0,37.0,18.0,26.0,28.0,3.0,24.0,7.0,10.0,10.0,7.0,14.0,7.0,7.0
05ocQutkURd6,93.34,14.0,3.0,11.0,42.0,15.0,20.0,28.0,0.0,5.0,0.0,3.0,0.0,3.0,2.0,2.0,6.0
06YUNBA9ZRLq,0.00,14.0,8.0,12.0,35.0,22.0,12.0,24.0,6.0,23.0,7.0,15.0,8.0,9.0,8.0,1.0,4.0


## Feature Selection for train_quant using random forests

In [7]:
# Make sure participant_id is a column, not the index
train_quant = train_quant.reset_index()
train_cat = train_cat.reset_index()
train_soln = train_soln.reset_index()
train_quant.head()

,participant_id,EHQ_EHQ_Total,ColorVision_CV_Score,APQ_P_APQ_P_CP,APQ_P_APQ_P_ID,APQ_P_APQ_P_INV,APQ_P_APQ_P_OPD,APQ_P_APQ_P_PM,APQ_P_APQ_P_PP,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial
0,00aIpNTbG5uh,100.00,13.0,3.0,15.0,44.0,14.0,20.0,27.0,3.0,17.0,4.0,11.0,5.0,8.0,6.0,2.0,9.0
1,00fV0OyyoLfw,92.27,14.0,3.0,12.0,35.0,25.0,28.0,30.0,5.0,20.0,4.0,13.0,5.0,8.0,7.0,3.0,8.0
2,04X1eiS79T4B,86.67,14.0,3.0,21.0,37.0,18.0,26.0,28.0,3.0,24.0,7.0,10.0,10.0,7.0,14.0,7.0,7.0
3,05ocQutkURd6,93.34,14.0,3.0,11.0,42.0,15.0,20.0,28.0,0.0,5.0,0.0,3.0,0.0,3.0,2.0,2.0,6.0
4,06YUNBA9ZRLq,0.00,14.0,8.0,12.0,35.0,22.0,12.0,24.0,6.0,23.0,7.0,15.0,8.0,9.0,8.0,1.0,4.0


In [8]:
train_cat.head()

,participant_id,Basic_Demos_Enroll_Year,Basic_Demos_Study_Site,PreInt_Demos_Fam_Child_Ethnicity,PreInt_Demos_Fam_Child_Race,MRI_Track_Scan_Location,Barratt_Barratt_P1_Edu,Barratt_Barratt_P1_Occ
0,00aIpNTbG5uh,2019,4,1.0,0.0,3.0,21.0,45.0
1,00fV0OyyoLfw,2017,1,0.0,9.0,2.0,21.0,0.0
2,04X1eiS79T4B,2017,1,1.0,2.0,2.0,9.0,0.0
3,05ocQutkURd6,2018,1,3.0,8.0,2.0,18.0,10.0
4,06YUNBA9ZRLq,2018,1,0.0,1.0,2.0,12.0,0.0


In [9]:
# Combine with targets temporarily
quant_with_targets = train_quant.merge(train_soln[['participant_id', 'ADHD_Outcome',
                                                   'Sex_F']], on='participant_id')

# Compute correlations
corrs = quant_with_targets.drop(columns='participant_id').corr()

# Grab correlation with targets
adhd_corr = corrs['ADHD_Outcome'].abs().sort_values(ascending=False)
sex_corr = corrs['Sex_F'].abs().sort_values(ascending=False)

# Keep top N or those above a threshold
top_adhd_feats = adhd_corr[adhd_corr > 0.05].index.tolist()
top_sex_feats = sex_corr[sex_corr > 0.05].index.tolist()

# Union of both sets, drop targets themselves if included
quant_selected = list(set(top_adhd_feats + top_sex_feats) - {'ADHD_Outcome', 'Sex_F'})
quant_selected

['SDQ_SDQ_Hyperactivity',
 'SDQ_SDQ_Internalizing',
 'SDQ_SDQ_Generating_Impact',
 'APQ_P_APQ_P_INV',
 'ColorVision_CV_Score',
 'APQ_P_APQ_P_CP',
 'APQ_P_APQ_P_ID',
 'SDQ_SDQ_Emotional_Problems',
 'SDQ_SDQ_Conduct_Problems',
 'SDQ_SDQ_Peer_Problems',
 'APQ_P_APQ_P_PP',
 'SDQ_SDQ_Prosocial',
 'SDQ_SDQ_Externalizing',
 'APQ_P_APQ_P_OPD',
 'SDQ_SDQ_Difficulties_Total']

In [10]:
from sklearn.ensemble import RandomForestClassifier

# Merge everything on participant_id
X_full = train_quant[['participant_id'] + quant_selected] \
    .merge(train_cat, on='participant_id') \
    .merge(train_func_pca, on='participant_id')

# Target labels
y = train_soln[['participant_id', 'ADHD_Outcome', 'Sex_F']]

# Align rows across X and y
X_full = X_full.merge(y, on='participant_id')
y = X_full[['ADHD_Outcome', 'Sex_F']]
X_full = X_full.drop(columns=['ADHD_Outcome', 'Sex_F'])

# Use combined target just for ranking features
combo_target = y['ADHD_Outcome'] + y['Sex_F']

# Train the Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_full.drop(columns=['participant_id']), combo_target)

# Rank feature importances
feat_importances = pd.Series(rf.feature_importances_, index=X_full.drop(columns=['participant_id']).columns)
top_features_final = feat_importances.sort_values(ascending=False).head(50).index.tolist()

# Final X for model training
X_selected = X_full[top_features_final]
X_selected

,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Externalizing,fmri_pca_50,SDQ_SDQ_Difficulties_Total,fmri_pca_48,fmri_pca_31,fmri_pca_37,fmri_pca_5,fmri_pca_35,fmri_pca_33,...,fmri_pca_24,fmri_pca_23,fmri_pca_42,fmri_pca_11,fmri_pca_32,fmri_pca_43,fmri_pca_29,fmri_pca_41,fmri_pca_12,fmri_pca_17
0,8.0,11.0,2.962495,17.0,-1.168673,1.145635,-1.590348,0.697012,2.182171,0.400605,...,-1.097761,-0.680154,0.943617,1.281826,6.149201,-1.060490,0.023601,0.750913,1.333274,-1.161215
1,8.0,13.0,1.211692,20.0,-0.427006,0.622077,-2.349019,-1.204589,2.248471,2.619120,...,-0.894607,-0.738267,0.486946,2.213616,0.224037,0.167213,-0.963297,-0.777104,-0.086101,-0.787268
2,7.0,10.0,-0.577178,24.0,-0.818002,1.623852,-2.521894,1.955186,-0.407818,-0.340320,...,-1.724498,-1.243957,2.345653,-1.466195,-2.476533,-1.025545,-2.224522,3.813841,3.939671,-4.886952
3,3.0,3.0,-0.215200,5.0,-1.276276,0.181463,0.652337,-3.212116,1.049282,-1.947907,...,-0.190874,1.904781,-0.358380,0.110044,-2.052666,2.735397,1.985295,1.013148,0.812512,-0.753405
4,9.0,15.0,-0.073863,23.0,-1.758016,0.042373,-0.122060,-0.906084,1.637541,-0.256547,...,-1.217281,-0.040720,-1.529037,-0.895010,-0.168207,0.604799,0.246489,1.424673,-1.794671,-2.317038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,8.0,13.0,-0.697615,20.0,1.664023,-1.857924,-0.901934,0.328386,-0.711845,-1.979503,...,0.486715,-1.325598,3.151765,-2.206764,2.085514,2.036356,-2.381402,0.610579,-1.016769,1.351153
1209,3.0,5.0,1.523171,5.0,-0.986513,1.243697,-0.313522,1.456239,0.622894,0.072495,...,-1.012149,1.459951,0.367025,0.847750,-2.279689,0.093522,0.102984,0.174177,1.487577,-0.703363
1210,7.0,9.0,-1.530661,14.0,-2.140699,-1.024107,2.274901,2.140782,1.598099,-1.626188,...,-0.866915,2.056705,-1.758260,1.057415,-2.228752,-1.749446,-0.637212,-1.554133,-1.445515,0.980649
1211,7.0,8.0,2.279119,10.0,2.218683,-2.341829,-5.416851,-1.452593,1.097705,2.229249,...,2.654675,4.084691,1.736748,1.646329,-0.709525,-2.210757,0.777506,-2.236938,-5.609986,0.775184


In [11]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)

# Convert to tensors
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y[['ADHD_Outcome', 'Sex_F']].values, dtype=torch.float32)

# Stratified train/test split
stratify_labels = y['ADHD_Outcome'].astype(str) + y['Sex_F'].astype(str)
X_train, X_test, y_train, y_test = train_test_split(
    X_tensor, y_tensor, test_size=0.2, random_state=42, stratify=stratify_labels)

train_ds = TensorDataset(X_train, y_train)
test_ds = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=32)

# Beefed-up model
class BetterMultiOutputNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.shared = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        self.head_adhd = nn.Linear(128, 1)
        self.head_sex = nn.Linear(128, 1)

    def forward(self, x):
        shared_out = self.shared(x)
        return torch.sigmoid(self.head_adhd(shared_out)), torch.sigmoid(self.head_sex(shared_out))

# Weighted BCE Loss with 2x weight for Female ADHD cases
class WeightedBCELoss(nn.Module):
    def __init__(self, weight_female_adhd=2):
        super(WeightedBCELoss, self).__init__()
        self.weight_female_adhd = weight_female_adhd

    def forward(self, predictions, targets, sex):
        # Calculate standard BCE loss
        bce_loss = nn.BCEWithLogitsLoss(reduction='none')(predictions, targets)
        
        # Apply weight for female ADHD cases (ADHD_Outcome=1, Sex_F=1)
        weight = torch.ones_like(bce_loss)
        weight[(targets == 1) & (sex == 1)] = self.weight_female_adhd
        
        # Apply the weights to the loss
        weighted_loss = weight * bce_loss
        return weighted_loss.mean()

# Initialize model and loss function
model = BetterMultiOutputNN(X_selected.shape[1])
weighted_criterion = WeightedBCELoss(weight_female_adhd=2)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.5)

# Training loop with early stopping
best_val_loss = np.inf
patience = 5
epochs_no_improve = 0

for epoch in range(30):
    model.train()
    running_loss = 0
    for xb, yb in train_loader:
        adhd_pred, sex_pred = model(xb)
        # Extract ADHD and Sex labels
        adhd_labels = yb[:, 0]
        sex_labels = yb[:, 1]
        
        # Compute weighted loss for both ADHD and Sex
        loss_adhd = weighted_criterion(adhd_pred.squeeze(), adhd_labels, sex_labels)
        loss_sex = weighted_criterion(sex_pred.squeeze(), sex_labels, sex_labels)
        
        # Total loss
        loss = loss_adhd + loss_sex
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validation
    model.eval()
    val_losses = []
    with torch.no_grad():
        for xb, yb in test_loader:
            adhd_pred, sex_pred = model(xb)
            loss_adhd = weighted_criterion(adhd_pred.squeeze(), yb[:, 0], yb[:, 1])
            loss_sex = weighted_criterion(sex_pred.squeeze(), yb[:, 1], yb[:, 1])
            val_loss = loss_adhd + loss_sex
            val_losses.append(val_loss.item())
    avg_val_loss = np.mean(val_losses)
    scheduler.step(avg_val_loss)

    print(f"Epoch {epoch+1} | Train Loss: {running_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_model = model.state_dict()
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print("Early stopping triggered.")
            break

# Evaluation
model.load_state_dict(best_model)
model.eval()
all_preds, all_true = [], []

with torch.no_grad():
    for xb, yb in test_loader:
        adhd_pred, sex_pred = model(xb)
        preds = torch.cat([adhd_pred, sex_pred], dim=1)
        all_preds.append(preds)
        all_true.append(yb)

all_preds = torch.cat(all_preds).numpy()
all_true = torch.cat(all_true).numpy()

# Thresholding
adhd_pred_labels = (all_preds[:, 0] > 0.5).astype(int)
sex_pred_labels = (all_preds[:, 1] > 0.5).astype(int)

# Ensure the true labels are correctly handled (indexing properly)
all_true_adhd = all_true[:, 0].astype(int)
all_true_sex = all_true[:, 1].astype(int)

# Compute weighted F1 Score with 2x weight for Female ADHD cases
def weighted_f1(y_true, y_pred, weight_column):
    # Assign a weight of 2x for Female ADHD cases
    weight = np.ones_like(y_true)
    weight[(y_true == 1) & (weight_column == 1)] = 3  # Apply 2x weight for Female ADHD
    
    f1 = f1_score(y_true, y_pred, average='weighted', sample_weight=weight)
    return f1

f1_adhd = weighted_f1(all_true_adhd, adhd_pred_labels, all_true_sex)
f1_sex = weighted_f1(all_true_sex, sex_pred_labels, all_true_sex)

print(f"\nADHD Weighted F1 Score: {f1_adhd:.4f}")
print(f"Sex Weighted F1 Score: {f1_sex:.4f}")

# Metrics for ADHD
def print_metrics(y_true, y_pred, name):
    print(f"\n {name} Metrics:")
    print(f"Accuracy:  {accuracy_score(y_true, y_pred):.4f}")
    print(f"F1 Score:  {f1_score(y_true, y_pred):.4f}")
    print(f"Precision: {precision_score(y_true, y_pred):.4f}")
    print(f"Recall:    {recall_score(y_true, y_pred):.4f}")

print_metrics(all_true_adhd, adhd_pred_labels, "ADHD")
print_metrics(all_true_sex, sex_pred_labels, "Sex")

Epoch 1 | Train Loss: 50.3702 | Val Loss: 1.5962
Epoch 2 | Train Loss: 48.3551 | Val Loss: 1.5582
Epoch 3 | Train Loss: 47.2033 | Val Loss: 1.5464
Epoch 4 | Train Loss: 46.4335 | Val Loss: 1.5311
Epoch 5 | Train Loss: 46.0317 | Val Loss: 1.5251
Epoch 6 | Train Loss: 45.3612 | Val Loss: 1.5341
Epoch 7 | Train Loss: 44.8930 | Val Loss: 1.5375
Epoch 8 | Train Loss: 44.6739 | Val Loss: 1.5355
Epoch 9 | Train Loss: 44.3715 | Val Loss: 1.5288
Epoch 10 | Train Loss: 44.0580 | Val Loss: 1.5379
Early stopping triggered.

ADHD Weighted F1 Score: 0.8064
Sex Weighted F1 Score: 0.4881

 ADHD Metrics:
Accuracy:  0.7654
F1 Score:  0.8412
Precision: 0.7865
Recall:    0.9042

 Sex Metrics:
Accuracy:  0.6296
F1 Score:  0.3750
Precision: 0.4426
Recall:    0.3253


## Testing and Submission

In [14]:
# Use read_excel and use participant_id as the index
test_cat = pd.read_excel("../../widsdatathon2025/TEST/TEST_CATEGORICAL.xlsx")
test_func = pd.read_csv("../../widsdatathon2025/TEST/TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv")
test_quant = pd.read_excel("../../widsdatathon2025/TEST/TEST_QUANTITATIVE_METADATA.xlsx")
solutions_path = "../../widsdatathon2025/TEST/TEST_SOLUTIONS.xlsx"

In [15]:
# Set index for merging
test_cat.set_index("participant_id", inplace=True)
test_quant.set_index("participant_id", inplace=True)
test_func.set_index("participant_id", inplace=True)

# train_cat.head()
train_func.head()
# train_quant.head()

,participant_id,0throw_1thcolumn,0throw_2thcolumn,0throw_3thcolumn,0throw_4thcolumn,0throw_5thcolumn,0throw_6thcolumn,0throw_7thcolumn,0throw_8thcolumn,0throw_9thcolumn,...,195throw_196thcolumn,195throw_197thcolumn,195throw_198thcolumn,195throw_199thcolumn,196throw_197thcolumn,196throw_198thcolumn,196throw_199thcolumn,197throw_198thcolumn,197throw_199thcolumn,198throw_199thcolumn
0,Cfwaf5FX7jWK,0.548480,0.713607,0.557319,0.524369,0.693364,0.770032,0.724406,0.390118,0.547912,...,0.080423,-0.054581,-0.088163,-0.028574,0.444847,0.350149,-0.012601,0.665750,0.560565,0.555732
1,vhGrzmvA3Hjq,0.427740,0.363022,0.402862,0.363003,0.534558,0.345347,0.409471,0.303328,0.402515,...,0.198009,-0.000724,0.083122,0.033043,0.687497,0.306229,0.717485,0.461809,0.559632,0.350027
2,ULliyEXjy4OV,0.139572,0.390106,-0.087041,0.196852,0.088148,0.023843,0.381782,0.068979,0.377488,...,0.051319,0.023630,-0.056819,0.117396,0.576086,0.517831,0.527044,0.605038,0.609856,0.750987
3,LZfeAb1xMtql,0.133561,0.778326,0.416355,0.471840,0.568460,0.633660,0.501113,0.345461,0.467943,...,0.046183,-0.238962,0.121868,-0.260970,0.646818,0.594902,0.608156,0.595459,0.683189,0.542296
4,EnFOUv0YK1RG,0.126699,0.575446,0.509422,0.363193,0.427544,0.449924,0.451796,0.223927,0.298248,...,0.315734,0.002234,0.290791,0.344149,0.480214,0.539824,0.447322,0.293088,0.148529,0.539823


In [16]:
from sklearn.decomposition import PCA

# Select only fMRI columns
fmri_data = test_func.drop(columns=['participant_id'], errors='ignore')

# Apply PCA to reduce to top N components (e.g. 50)
pca = PCA(n_components=50)
fmri_pca = pca.fit_transform(fmri_data)

test_func_pca = pd.DataFrame(fmri_pca, index=test_func.index,
                           columns=[f'fmri_pca_{i}' for i in range(1, 51)])
test_func_pca.head()

# Check missing values *before* merging
missing = test_quant.isnull().sum()
missing_cat = test_cat.isnull().sum()
print("Missing values in train_quant:\n", missing[missing > 0])
print("Missing values in train_cat:\n", missing_cat[missing_cat > 0])

# Define columns with small gaps to impute
small_gap_cols = [
    'EHQ_EHQ_Total', 'ColorVision_CV_Score',
    'APQ_P_APQ_P_CP', 'APQ_P_APQ_P_ID', 'APQ_P_APQ_P_INV',
    'APQ_P_APQ_P_OPD', 'APQ_P_APQ_P_PM', 'APQ_P_APQ_P_PP',
    'SDQ_SDQ_Conduct_Problems', 'SDQ_SDQ_Difficulties_Total',
    'SDQ_SDQ_Emotional_Problems', 'SDQ_SDQ_Externalizing',
    'SDQ_SDQ_Generating_Impact', 'SDQ_SDQ_Hyperactivity',
    'SDQ_SDQ_Internalizing', 'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Prosocial',
    'PreInt_Demos_Fam_Child_Ethnicity', 'PreInt_Demos_Fam_Child_Race',
    'MRI_Track_Scan_Location'
]

small_gap_cols_cat = [
    'Barratt_Barratt_P1_Edu', 'Barratt_Barratt_P1_Occ'
]

# Fill small gaps using column means
for col in small_gap_cols:
    if col in test_quant.columns:
        test_quant[col] = test_quant[col].fillna(test_quant[col].mean())
for col in small_gap_cols_cat:
    if col in test_cat.columns:
        test_cat[col] = test_cat[col].fillna(test_cat[col].mean())

test_quant.drop(columns=['MRI_Track_Age_at_Scan'], inplace=True) # Dropped
test_cat.drop(columns=['Barratt_Barratt_P2_Occ', 'Barratt_Barratt_P2_Edu'], 
              inplace=True) # Dropped

# Make sure participant_id is a column, not the index
test_quant = test_quant.reset_index()
test_cat = test_cat.reset_index()

Missing values in train_quant:
 EHQ_EHQ_Total                  1
ColorVision_CV_Score           9
APQ_P_APQ_P_CP                15
APQ_P_APQ_P_ID                15
APQ_P_APQ_P_INV               15
APQ_P_APQ_P_OPD               15
APQ_P_APQ_P_PM                15
APQ_P_APQ_P_PP                15
SDQ_SDQ_Conduct_Problems      30
SDQ_SDQ_Difficulties_Total    30
SDQ_SDQ_Emotional_Problems    30
SDQ_SDQ_Externalizing         30
SDQ_SDQ_Generating_Impact     30
SDQ_SDQ_Hyperactivity         30
SDQ_SDQ_Internalizing         30
SDQ_SDQ_Peer_Problems         30
SDQ_SDQ_Prosocial             30
dtype: int64
Missing values in train_cat:
 PreInt_Demos_Fam_Child_Ethnicity     3
PreInt_Demos_Fam_Child_Race          6
Barratt_Barratt_P1_Edu               1
Barratt_Barratt_P1_Occ               1
Barratt_Barratt_P2_Edu              36
Barratt_Barratt_P2_Occ              42
dtype: int64


In [17]:
# Merge everything on participant_id
X_full = test_quant[['participant_id'] + quant_selected] \
    .merge(test_cat, on='participant_id') \
    .merge(test_func_pca, on='participant_id')

# Final X for model training (top_features_final from random forests done on training set)
X_test_selected = X_full[top_features_final]
X_test_selected

,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Externalizing,fmri_pca_50,SDQ_SDQ_Difficulties_Total,fmri_pca_48,fmri_pca_31,fmri_pca_37,fmri_pca_5,fmri_pca_35,fmri_pca_33,...,fmri_pca_24,fmri_pca_23,fmri_pca_42,fmri_pca_11,fmri_pca_32,fmri_pca_43,fmri_pca_29,fmri_pca_41,fmri_pca_12,fmri_pca_17
0,7.0,9.0,-0.324290,12.0,-2.324203,1.942373,-1.688586,0.930748,-1.173564,0.180032,...,1.275498,-0.306033,1.716070,4.150425,0.379226,-0.773796,2.754719,-0.926181,-1.461191,-1.520688
1,3.0,5.0,-1.979338,16.0,3.631067,1.124477,0.380299,2.667334,-4.309650,-1.263532,...,3.491815,-1.864578,0.953023,4.199631,1.192092,0.447701,-0.215396,-2.004347,1.631035,1.977907
2,5.0,6.0,0.759340,7.0,-1.706108,1.724166,-0.376089,-2.053957,0.789639,-2.873811,...,0.315485,-1.639236,0.413656,-3.036462,1.784396,1.916834,0.875269,-1.200967,-1.393299,2.840884
3,6.0,10.0,0.383404,15.0,1.082908,0.890923,-2.085292,0.741584,0.295368,0.601205,...,-0.769030,0.028741,-2.030191,-4.397754,-0.300261,0.928140,-1.604671,-0.607864,-1.457293,0.699531
4,10.0,12.0,-0.581088,18.0,0.865579,-1.806361,1.162843,-2.280240,1.690596,0.055766,...,1.480784,-0.838887,-0.719495,-0.571178,-2.855215,2.524235,1.205504,-0.436673,1.126602,0.598251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,6.0,7.0,1.779283,18.0,-0.912265,-0.761078,-0.543698,1.465643,-0.862118,1.494788,...,-0.612311,0.887675,2.123764,1.626029,-2.169787,-0.058980,0.829895,-0.760154,-2.224182,0.036516
300,9.0,11.0,-0.496668,16.0,2.589713,-1.110183,-0.712363,1.268605,7.740441,2.699661,...,6.884819,2.536611,0.035010,1.863492,1.051392,-2.556971,2.006615,-2.112862,-4.859356,-0.304729
301,3.0,4.0,-1.422683,11.0,0.148043,3.704947,1.921459,2.498292,-1.939094,0.587137,...,1.781372,2.695057,-1.528450,1.438116,-0.957751,1.887081,3.482524,3.508512,-1.523458,-1.482218
302,5.0,10.0,1.290901,21.0,-1.170213,3.771796,1.804700,-6.782507,0.371130,-0.162611,...,0.782527,-0.035039,-0.616927,-0.129434,2.974451,-0.136344,-2.643271,-0.928147,-1.844048,0.866393


In [20]:
# Standardize test data
X_test_scaled = scaler.transform(X_test_selected)

# Convert to PyTorch tensor
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

# Load into DataLoader
test_loader_final = DataLoader(X_test_tensor, batch_size=32)

# Set model to eval mode and load best weights
model.load_state_dict(best_model)
model.eval()

adhd_preds, sex_preds = [], []

with torch.no_grad():
    for xb in test_loader_final:
        adhd_out, sex_out = model(xb)
        adhd_preds.append(adhd_out)
        sex_preds.append(sex_out)

# Concatenate and threshold at 0.5
adhd_preds = torch.cat(adhd_preds).squeeze().numpy()
sex_preds = torch.cat(sex_preds).squeeze().numpy()

adhd_pred_labels = (adhd_preds > 0.5).astype(int)
sex_pred_labels = (sex_preds > 0.5).astype(int)

# Create test_soln submission DataFrame
test_soln = pd.DataFrame({
    'participant_id': X_full['participant_id'],  # or test_func['participant_id']
    'ADHD_Outcome': adhd_pred_labels,
    'Sex_F': sex_pred_labels
})

test_soln.to_csv("submission.csv", index=False)
test_soln

,participant_id,ADHD_Outcome,Sex_F
0,Cfwaf5FX7jWK,1,0
1,vhGrzmvA3Hjq,1,1
2,ULliyEXjy4OV,1,0
3,LZfeAb1xMtql,1,0
4,EnFOUv0YK1RG,1,0
...,...,...,...
299,UadZfjdEg7eG,1,0
300,IUEHiLmQAqCi,1,1
301,cRySmCadYFRO,1,0
302,E3MvDUtJadc5,1,0
